In [2]:
import os
import sys
import torch
import torch.nn as nn

torch_path = "C:/libtorch/lib"
os.add_dll_directory(torch_path)

atlas_path = "C:/Users/natha/OneDrive/Desktop/C++/Atlas/x64/Release"
sys.path.append(atlas_path)


from AtlasPy.core import Hydra, Portfolio, Strategy
from AtlasPy.ast import *
from AtlasPy.model import *

In [3]:
exchange_path = "C:/Users/natha/OneDrive/Desktop/C++/Atlas/AtlasPy/src/exchangeVBT"
strategy_id = "test_strategy"
exchange_id = "test_exchange"
portfolio_id = "test_portfolio"

hydra = Hydra()
intial_cash = 100.0
exchange = hydra.addExchange(exchange_id, exchange_path, "")
portfolio = hydra.addPortfolio(portfolio_id, exchange, intial_cash)
hydra.build()

In [4]:
class LinearRegressionModel(nn.Module):
    def __init__(self, input_size):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_size, 1)

    def forward(self, x1, x2):
        return self.linear(x1) + self.linear(x2)

my_module = LinearRegressionModel(3)
sm = torch.jit.script(my_module)
sm.save("linear_regression.pt")

In [5]:
n = 2
close = AssetReadNode.make("Close", 0, exchange)
open = AssetReadNode.make("Open", 0, exchange)
prev_close = AssetReadNode.make("Close", -1, exchange)
torch_script_file = r'C:\Users\natha\OneDrive\Desktop\C++\Atlas\AtlasPy\src\scripts\linear_regression.pt'

feat1 = AssetOpNode.make(
    close,
    prev_close,
    AssetOpType.SUBTRACT
)
feat2 = AssetOpNode.make(
    close,
    open,
    AssetOpType.SUBTRACT
)
target = ModelTarget(
    close,
    ModelTargetType.ABSOLUTE,
    2
)
config = ModelConfig(
    training_window = 5,
    walk_forward_window = 3,
    model_type = ModelType.TORCH,
    exchange = exchange,
)
torch_config = TorchModelConfig(
    config,
    torch_script_file = torch_script_file
)
torch_model = TorchModel(
    "torch_model",
    [feat1, feat2],
    target,
    torch_config
)
exchange.registerModel(torch_model)

In [ ]:
for i in range(6):
    hydra.step()

: 